<a href="https://colab.research.google.com/github/priyangshu-datta/master-project/blob/main/llm_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install datasets chromadb transformers auto-gptq einops optimum accelerate bitsandbytes sentence_transformers InstructorEmbedding

In [ ]:
from datasets import Dataset

In [ ]:
DMDD = Dataset.load_from_disk("./drive/MyDrive/Thesis/Dataset_Mention_Extraction/data/DMDD_llm_rag/")
DMDD = DMDD.to_iterable_dataset()
# Start from: 4151

In [ ]:
import re
import string

def split_sentences(text, max_tokens=100):
  matches = re.finditer(r"\.\s[A-Z]", text)
  start = 0
  sentences = []

  sentence = ""
  tokens=0
  for match in matches:
      end = match.start() + 1
      sentence += text[start:end]
      tokens += len(list(filter(is_word, sentence.split(" "))))
      if (tokens > max_tokens):
          sentences.append(sentence.strip())
          sentence = ""
          tokens = 0
      start = match.end() - 2
  sentences.append(text[start:])
  return sentences

def is_word(word):
    return len(word) > 3 and word not in string.punctuation.split()

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

In [ ]:
chroma_client = chromadb.PersistentClient(path="./drive/MyDrive/Thesis/Dataset_Mention_Extraction/vector_db")
collection_name = "paper_paras"
model_name = "allenai/scibert_scivocab_uncased"
collection = chroma_client.get_or_create_collection(
    collection_name,
    embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name=model_name, device="cuda"
    )
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/437 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/NeuralBeagle14-7B-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

In [ ]:
context = " ".join(collection.query(include=["documents"], query_texts="what are the name of datasets used in the papers?")['documents'][0])
query = "What are the name of datasets used in the papers?"

meta_prompt = f"""
instruction: You are an avid reader of research papers. You will be provided with multiple paragraphs as context, and a query. You will try to answer the query in minimum number of tokens. Do not try to forcefully answer, only answer from the context provided.
context: {context}
query: {query}
answer:
"""

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(meta_prompt)[0]['generated_text'])


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(



instruction: You are an avid reader of research papers. You will be provided with multiple paragraphs as context, and a query. You will try to answer the query in minimum number of tokens. Do not try to forcefully answer, only answer from the context provided.
context: Third, the images are not scraped based on a predefined list of class names or tags, leading to natural class statistics and avoiding the initial design bias on what should be in the dataset. The ∼9 million images in the Open Images Dataset were collected using the following procedure: 1. Identify all Flickr images with CC-BY license. This was done in November 2015. 2. As our dataset is fundamentally different from the prior datasets such as Geoquery and WikiSQL, we adapted these models to our task as follows. We created a 'big' column list by concatenating columns in all tables of the database together as 4 Please check our website for the latest updates on the task at https://yale-lily.github.io/spider What is the num

In [ ]:
from tqdm import tqdm

def insert_in_db(dataset, start):
    for i, paper in tqdm(enumerate(dataset)):
        if i < start :
            continue
        documents = split_sentences(paper['text'])
        metadatas = [{**paper['meta_data'], "id": paper['id']}]*len(documents)
        ids = [f"{paper['id']}->{i}" for i in range(len(documents))]
        collection.add(
            documents=documents,
            metadatas=metadatas,
            ids=ids,
        )

In [1]:
%%capture
%pip install datasets chromadb transformers auto-gptq einops optimum accelerate bitsandbytes sentence_transformers InstructorEmbedding

In [2]:
import json
from google.colab import drive

drive.mount('/content/drive')

with open("./drive/MyDrive/Thesis/Dataset_Mention_Extraction/data/dmdd-eval.json", "r") as file:
    json_dict = json.load(file)

Mounted at /content/drive


In [3]:
import re
import string

def split_sentences(text, max_tokens=50):
  matches = re.finditer(r"\.\s[A-Z]", text)
  start = 0
  sentences = []

  sentence = ""
  tokens=0
  for match in matches:
      end = match.start() + 1
      sentence += text[start:end]
      tokens += len(list(filter(is_word, sentence.split(" "))))
      if (tokens > max_tokens):
          sentences.append(sentence.strip())
          sentence = ""
          tokens = 0
      start = match.end() - 2
  sentences.append(text[start:])
  return sentences

def is_word(word):
    return len(word) > 3 and word not in string.punctuation.split()

In [4]:
import chromadb
from chromadb.utils import embedding_functions
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from pprint import pprint

def connect_to_db():
  chroma_client = chromadb.Client()
  collection_name = "temp"
  model_name = "allenai/scibert_scivocab_uncased"
  collection = chroma_client.get_or_create_collection(
      collection_name,
      embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction(
          model_name=model_name, device="cuda"
      ),
      metadata={"hnsw:space": "cosine"}
  )
  return chroma_client, collection

def fill_db(chunks, collection):
  collection.add(documents=chunks, ids=list(map(str,range(len(chunks)))))

def clear_db(client):
  client.delete_collection("temp")

def prepare_llm():
  model_name_or_path = "TheBloke/dolphin-2.6-mistral-7B-dpo-laser-GPTQ"
  #  "TheBloke/NeuralBeagle14-7B-GPTQ"
  model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                              device_map="auto",
                                              trust_remote_code=False,
                                              revision="main")

  tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
  return model, tokenizer


def extract_dataset_mentions(text):
  try:
    chroma_client, collection = connect_to_db()
    chunks = split_sentences(text)
    fill_db(chunks, collection)

    model, tokenizer = prepare_llm()

    docs = collection.query(include=["documents"],
                            query_texts="What are the name of datasets used in the papers?")['documents'][0]
    pprint(docs)

    query = "What are the name of datasets given in the context?"

    context = " ".join(docs)
    instruction = """You will be provided with snippets from a research paper.
A dataset is a source of data used to experiment on. Datasets normally have names which are nouns and not dictionary words. But their are excpetions.
Try to understand the context of the sentences to extract any and all datasets whether explicitly mentioned or just implied in the text.
Return the extracted dataset names in a list. Use the least number of words possible.
If you cannot identify any datasets, do not pretend you know it, but say"I don't know"."""
    meta_prompt = f"""<|im_start|>system
{instruction}<|im_end|>
<|im_start|>user
{context}
<|im_end|>
<|im_start|>assistant
The datasets are:
"""

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.1,
        top_p=0.2,
        top_k=100,
        repetition_penalty=1.2
    )

    print(pipe(meta_prompt)[0]['generated_text'])
  except:
    print("Error")
  finally:
    clear_db(chroma_client)

In [5]:
def eval(id):
  text = json_dict[f'{id}']['text']
  observed_datasets = extract_dataset_mentions(text)
  actual_datasets = set(json_dict[f'{id}']['dataset_names'])
  print(actual_datasets)
  # observed_datasets = set(observed_datasets)
  # if len(actual_datasets) > 0 and len(observed_datasets) == 0 : return 0
  # count = 0
  #
  # for d in actual_datasets:
  #   if d in observed_datasets:
  #     count+=1
  # return count / len(actual_datasets)

In [14]:
eval(3)
# json_dict['0']['dataset_names']

['However, in practice, there are application- dependent reasons that make '
 'this assumption to fail. The consequence is that the classifier performs '
 'worse than expected. In this case, we have to augment the machine learning '
 'formalism with the concepts of source and target domains.',
 'To set the hyper- parameters of the different adaptation methods( i.e. the '
 "C's), we used the Virtual- INRIA DA problem. These values are then used for "
 'the rest of experiments. In order to show the effectiveness of the proposed '
 'RF- DA methods, we run experiments for different combinations of source and '
 'target domains.',
 'For INRIA and KITTI this is ∼ 10%, while for Daimler we use ∼ 5% since it is '
 'a larger training dataset. Regarding the training of the RF- LE, we follow '
 'the instructions reported in[ 7], e.g. we use 100 trees and the maximum '
 'depth for the trees is set to 7, and HOG+LBP features are used as image '
 'descriptors. The average miss rate( AMR) vs false po

In [7]:
#@title Configure Gemini API key

import google.generativeai as genai
from google.colab import userdata

gemini_api_secret_name = 'GOOGLE_API_KEY'  # @param {type: "string"}

try:
  GOOGLE_API_KEY=userdata.get(gemini_api_secret_name)
  genai.configure(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
   print(f'Secret not found\n\nThis expects you to create a secret named {gemini_api_secret_name} in Colab\n\nVisit https://makersuite.google.com/app/apikey to create an API key\n\nStore that in the secrets section on the left side of the notebook (key icon)\n\nName the secret {gemini_api_secret_name}')
   raise e
except userdata.NotebookAccessError as e:
  print(f'You need to grant this notebook access to the {gemini_api_secret_name} secret in order for the notebook to access Gemini on your behalf.')
  raise e
except Exception as e:
  # unknown error
  print(f"There was an unknown error. Ensure you have a secret {gemini_api_secret_name} stored in Colab and it's a valid key from https://makersuite.google.com/app/apikey")
  raise e

In [15]:
#connect to the API and send an example message

text = "\u003C|im_start|>system You will be provided with snippets from a research paper. A dataset is a source of data used to experiment on. Datasets normally have names which are nouns and not dictionary words. But their are excpetions. Try to understand the context of the sentences to extract any and all datasets whether explicitly mentioned or just implied in the text. Return the extracted dataset names in a list. Use the least number of words possible. If you cannot identify any datasets, do not pretend you know it, but say\"I don't know\".\u003C|im_end|> \u003C|im_start|>user However, in practice, there are application- dependent reasons that make this assumption to fail. The consequence is that the classifier performs worse than expected. In this case, we have to augment the machine learning formalism with the concepts of source and target domains. To set the hyper- parameters of the different adaptation methods( i.e. the C's), we used the Virtual- INRIA DA problem. These values are then used for the rest of experiments. In order to show the effectiveness of the proposed RF- DA methods, we run experiments for different combinations of source and target domains. For INRIA and KITTI this is \u223C 10%, while for Daimler we use \u223C 5% since it is a larger training dataset. Regarding the training of the RF- LE, we follow the instructions reported in[ 7], e.g. we use 100 trees and the maximum depth for the trees is set to 7, and HOG+LBP features are used as image descriptors. The average miss rate( AMR) vs false positives per image( FPPI)[ 25] to report detection accuracy. We can find recent paradigmatic applications such as its use for pose recognition in depth images( Kinect)[ 4], or as classification layer in the GoogLeNet deep CNN architecture for large- scale image classification[ 5]. Usually, when learning any classifier, including RF, it is assumed that the training and testing samples follow the same probability distribution. The Virtual dataset is used only as source domain. The rest are considered both as source and target domains thanks to the use of different training and testing sets per dataset. In order to run a DA experiment we select a relatively low number of the samples available in the training set of the target- domain dataset. Vezhnevets and Buhmann[ 12] performed DA for the so- called semantic texton forest used for semantic segmentation. They consider the same theoretical setting as[ 11] but using L 1- norm for calculating the distance between two histograms instead of the chi- square. All these RF- DA methods require the source- domain samples, i.e. they follow the feature- transfer paradigm. Each train- test experiment is repeated five times and we report the mean and standard deviation of AMR versus FPPI in the interval of[ 0.01, 1]. We compare Node- Adapt, Path- Adapt and Tree- Adapt methods in each experiment with the RF trained only with source data( Src), the RF trained with all samples of the target domain( Tar 100%), and with the RF trained only on the limited number of target- domain samples(TarX%). This is done by solving the following optimization problem formulated for any node j( for the root j= 1 \u2227 S ta 1= S ta): where y k is the class label of the target- domain feature vector v ta k. Note that class labels do not change from source to target domains, so we do not need to distinguish domains for them. The main th tree are reached. The conclusion is that, although these three proposals achieve DA, Tree- Adapt is the most successful method to adapt RF- LE models, and opens the door to the use of virtual worlds as source domain for reducing the number of object annotations. As future work we plan several working lines. However, TreeAdapt still is the most effective method. On the other hand, for low memory adaptive embedded systems( e.g. a domestic cleaning robot), Path- Adapt is interesting in the sense that it requires scarce information from the source domain since the data and node hyper- planes are not required, just the source tree topology and the compact path SVMs are needed. So the system can come with a sort of common knowledge and be(self-)adapted for the particular environment. \u003C|im_end|> \u003C|im_start|>assistant The datasets are:" #@param {type: 'string'}

model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])

response = chat.send_message(text)
response.text

'Virtual-INRIA, Daimler, KITTI, Kinect, googLeNet'